In [1]:
import pandas as pd
import numpy as np
import phik
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, KFold, StratifiedKFold
from scipy.stats import uniform, randint
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import shap

import featuretools as ft
import xgboost as xgb

pd.set_option('display.max_columns', None)

In [2]:
pd.set_option('display.max_colwidth', None)

task = pd.read_excel("/Users/kewenyang/Documents/GitHub/Maybank_Classification/data/Assessment.xlsx",
                       engine='openpyxl',
                       sheet_name=0)
task

,Business Problem
0,The bank is always interested to increase customer base in different segments to up sell or X-sell the products and as part of it they want to upgrade the segment of the ETB(Exsisting To Bank) customers from Normal to Affluent for Upselling the relevant products to the prospective customers and help to increase bank revenue. So as a senior data scientist need to identify the hidden affluent customers from the Data for the bank to approach them for upgrading their segment.
1,NaN
2,Instructions:
3,1. Analysis of the data
4,2. Data Processing techniques if any
5,3.Modelling Approach(Modelling techniques used atleast 2 different modelling techniques and their comparision on different Model Evaluation metrics)
6,4.Coding scripts along with output(Jupyter notebook with the outputs included). Please Use Python for coding.
7,5.Summary of the Analysis and Business recommendations if any
8,"6. Explain the high level Mlops steps on productionalizing the Model output, Monitoring and CI/CD of the model"
9,7. Provide all the above requested Information in presentation slides except for the code scripts


In [3]:
pd.set_option('display.max_colwidth', None)

meta = pd.read_excel("/Users/kewenyang/Documents/GitHub/Maybank_Classification/data/Assessment.xlsx",
                       engine='openpyxl',
                       sheet_name=2)
meta

,Feature,Definition,Remarks
0,C_ID,Dummy customer ID,NaN
1,C_AGE,customer Age,NaN
2,C_EDU,customer Education,NaN
3,C_HSE,customer house type,NaN
4,PC,dummy postal code,NaN
5,INCM_TYP,income level code,NaN
6,gn_occ,occupation,NaN
7,NUM_PRD,number of distinct products held,"eg. Count of Savings, TD,Investment,Insurance etc."
8,CASATD_CNT,number of Current/savings(CASA) and Time deposit accounts held,NaN
9,MTHCASA,average monthly balance in CASA,NaN


In [4]:
df_raw = pd.read_excel("/Users/kewenyang/Documents/GitHub/Maybank_Classification/data/Assessment.xlsx",
                       engine='openpyxl',
                       sheet_name=1)
df_raw.head()

,C_ID,C_AGE,C_EDU,C_HSE,PC,INCM_TYP,gn_occ,NUM_PRD,CASATD_CNT,MTHCASA,MAXCASA,MINCASA,DRvCR,MTHTD,MAXTD,Asset value,HL_tag,AL_tag,pur_price_avg,UT_AVE,MAXUT,N_FUNDS,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,AVG_TRN_AMT,ANN_TRN_AMT,ANN_N_TRX,CC_LMT,C_seg
0,1443,65,Masters,EXECUTIVE CONDOMINIUM,19250.0,6.0,PMEB,3,8.0,6896.91,4899.08,910.88,1.020768e+06,105000.00,25000.00,111896.91,NaN,NaN,NaN,NaN,NaN,NaN,13.233333,NaN,NaN,NaN,NaN,NaN,34500.0,AFFLUENT
1,1559,86,O-Levels,PRIVATE CONDOMINIUM,99018.0,2.0,PMEB,4,13.0,51714.78,35740.55,1318.25,8.326420e+00,575571.97,135026.15,627286.75,NaN,NaN,NaN,NaN,NaN,NaN,727.629167,8530.88,273.44,2296.713333,27560.56,88.0,4000.0,AFFLUENT
2,1913,69,A-Levels,NaN,10155.0,3.0,PMEB,4,1.0,5420.09,5420.09,5420.09,4.106600e-01,NaN,NaN,64161.74,NaN,NaN,NaN,58424.550000,59600.88,1.0,367.389167,523.35,122.13,283.580833,3402.97,78.0,5000.0,AFFLUENT
3,2032,70,NaN,NaN,21863.0,2.0,RETIREE,3,1.0,13596.65,13596.65,13596.65,5.397180e+03,NaN,NaN,39792.30,NaN,NaN,NaN,28863.181667,33902.58,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,AFFLUENT
4,2048,72,O-Levels,EXECUTIVE CONDOMINIUM,10641.0,NaN,BLUE COLLAR,4,2.0,51639.18,32689.93,18949.25,3.607580e+03,NaN,NaN,67258.47,NaN,NaN,NaN,15060.339167,15619.29,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,AFFLUENT


In [5]:
# check target var distribution --> imbalance
df_raw.C_seg.value_counts()

C_seg
NORMAL      55157
AFFLUENT    10926
Name: count, dtype: int64

In [6]:
# C_ID is not unique
df_raw.loc[df_raw.C_ID == 59688]

,C_ID,C_AGE,C_EDU,C_HSE,PC,INCM_TYP,gn_occ,NUM_PRD,CASATD_CNT,MTHCASA,MAXCASA,MINCASA,DRvCR,MTHTD,MAXTD,Asset value,HL_tag,AL_tag,pur_price_avg,UT_AVE,MAXUT,N_FUNDS,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,AVG_TRN_AMT,ANN_TRN_AMT,ANN_N_TRX,CC_LMT,C_seg
7716,59688,41,NaN,NaN,11014.0,NaN,PMEB,4,2.0,8736.08,8736.08,8736.08,51293.240000,50000.0,50000.0,58736.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,AFFLUENT
9404,59688,63,NaN,NaN,31059.0,NaN,RETIREE,2,2.0,28889.66,28889.66,28889.66,274.533333,150000.0,150000.0,178889.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AFFLUENT
18364,59688,41,Degree,HDB 4-5 ROOM,80211.0,4.0,PMEB,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NORMAL
33573,59688,64,Degree,HDB 4-5 ROOM,50142.0,6.0,PMEB,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18700.0,NORMAL
37016,59688,18,NaN,NaN,89093.0,NaN,STUDENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NORMAL
62793,59688,38,NaN,NaN,51445.0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL


In [7]:
# drop original C_ID and replace it by row index
df_raw.drop("C_ID", axis=1, inplace=True)
df_raw.reset_index(drop=True, inplace=True)
df_raw = df_raw.reset_index()
df_raw.rename(columns={"index": "C_ID"}, inplace=True)
df_raw.head()

,C_ID,C_AGE,C_EDU,C_HSE,PC,INCM_TYP,gn_occ,NUM_PRD,CASATD_CNT,MTHCASA,MAXCASA,MINCASA,DRvCR,MTHTD,MAXTD,Asset value,HL_tag,AL_tag,pur_price_avg,UT_AVE,MAXUT,N_FUNDS,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,AVG_TRN_AMT,ANN_TRN_AMT,ANN_N_TRX,CC_LMT,C_seg
0,0,65,Masters,EXECUTIVE CONDOMINIUM,19250.0,6.0,PMEB,3,8.0,6896.91,4899.08,910.88,1.020768e+06,105000.00,25000.00,111896.91,NaN,NaN,NaN,NaN,NaN,NaN,13.233333,NaN,NaN,NaN,NaN,NaN,34500.0,AFFLUENT
1,1,86,O-Levels,PRIVATE CONDOMINIUM,99018.0,2.0,PMEB,4,13.0,51714.78,35740.55,1318.25,8.326420e+00,575571.97,135026.15,627286.75,NaN,NaN,NaN,NaN,NaN,NaN,727.629167,8530.88,273.44,2296.713333,27560.56,88.0,4000.0,AFFLUENT
2,2,69,A-Levels,NaN,10155.0,3.0,PMEB,4,1.0,5420.09,5420.09,5420.09,4.106600e-01,NaN,NaN,64161.74,NaN,NaN,NaN,58424.550000,59600.88,1.0,367.389167,523.35,122.13,283.580833,3402.97,78.0,5000.0,AFFLUENT
3,3,70,NaN,NaN,21863.0,2.0,RETIREE,3,1.0,13596.65,13596.65,13596.65,5.397180e+03,NaN,NaN,39792.30,NaN,NaN,NaN,28863.181667,33902.58,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,AFFLUENT
4,4,72,O-Levels,EXECUTIVE CONDOMINIUM,10641.0,NaN,BLUE COLLAR,4,2.0,51639.18,32689.93,18949.25,3.607580e+03,NaN,NaN,67258.47,NaN,NaN,NaN,15060.339167,15619.29,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,AFFLUENT


In [8]:
# check col with more missing values
missing = df_raw.isnull().any(axis=0)
missing = missing.loc[missing == True].index
missing

Index(['C_EDU', 'C_HSE', 'PC', 'INCM_TYP', 'gn_occ', 'CASATD_CNT', 'MTHCASA',
       'MAXCASA', 'MINCASA', 'DRvCR', 'MTHTD', 'MAXTD', 'HL_tag', 'AL_tag',
       'pur_price_avg', 'UT_AVE', 'MAXUT', 'N_FUNDS', 'CC_AVE',
       'MAX_MTH_TRN_AMT', 'MIN_MTH_TRN_AMT', 'AVG_TRN_AMT', 'ANN_TRN_AMT',
       'ANN_N_TRX', 'CC_LMT'],
      dtype='object')

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66083 entries, 0 to 66082
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   C_ID             66083 non-null  int64  
 1   C_AGE            66083 non-null  int64  
 2   C_EDU            27619 non-null  object 
 3   C_HSE            22167 non-null  object 
 4   PC               65597 non-null  float64
 5   INCM_TYP         36617 non-null  float64
 6   gn_occ           65205 non-null  object 
 7   NUM_PRD          66083 non-null  int64  
 8   CASATD_CNT       40991 non-null  float64
 9   MTHCASA          39045 non-null  float64
 10  MAXCASA          39045 non-null  float64
 11  MINCASA          39045 non-null  float64
 12  DRvCR            29708 non-null  float64
 13  MTHTD            14100 non-null  float64
 14  MAXTD            14100 non-null  float64
 15  Asset value      66083 non-null  float64
 16  HL_tag           2368 non-null   float64
 17  AL_tag      

In [12]:
drop_cols = ["HL_tag", "AL_tag", "pur_price_avg", "UT_AVE", "MAXUT", "N_FUNDS"]
df_raw.drop(drop_cols, axis=1, inplace=True)
df_raw.head(2)

,C_ID,C_AGE,C_EDU,C_HSE,PC,INCM_TYP,gn_occ,NUM_PRD,CASATD_CNT,MTHCASA,MAXCASA,MINCASA,DRvCR,MTHTD,MAXTD,Asset value,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,AVG_TRN_AMT,ANN_TRN_AMT,ANN_N_TRX,CC_LMT,C_seg
0,0,65,Masters,EXECUTIVE CONDOMINIUM,19250.0,6.0,PMEB,3,8.0,6896.91,4899.08,910.88,1.020768e+06,105000.00,25000.00,111896.91,13.233333,NaN,NaN,NaN,NaN,NaN,34500.0,AFFLUENT
1,1,86,O-Levels,PRIVATE CONDOMINIUM,99018.0,2.0,PMEB,4,13.0,51714.78,35740.55,1318.25,8.326420e+00,575571.97,135026.15,627286.75,727.629167,8530.88,273.44,2296.713333,27560.56,88.0,4000.0,AFFLUENT


In [36]:
from sklearn.impute import SimpleImputer

categorical = ["C_EDU", "C_HSE", "gn_occ", "C_seg"]

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df_cat = pd.DataFrame(imp.fit_transform(df_raw.loc[:, categorical]), columns=categorical)
df_cat.head()

,C_EDU,C_HSE,gn_occ,C_seg
0,Masters,EXECUTIVE CONDOMINIUM,PMEB,AFFLUENT
1,O-Levels,PRIVATE CONDOMINIUM,PMEB,AFFLUENT
2,A-Levels,HDB 4-5 ROOM,PMEB,AFFLUENT
3,Degree,HDB 4-5 ROOM,RETIREE,AFFLUENT
4,O-Levels,EXECUTIVE CONDOMINIUM,BLUE COLLAR,AFFLUENT


In [39]:
non_cat

['C_ID',
 'C_AGE',
 'PC',
 'INCM_TYP',
 'NUM_PRD',
 'CASATD_CNT',
 'MTHCASA',
 'MAXCASA',
 'MINCASA',
 'DRvCR',
 'MTHTD',
 'MAXTD',
 'Asset value',
 'CC_AVE',
 'MAX_MTH_TRN_AMT',
 'MIN_MTH_TRN_AMT',
 'AVG_TRN_AMT',
 'ANN_TRN_AMT',
 'ANN_N_TRX',
 'CC_LMT']

In [40]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
non_cat = [col for col in df_raw.columns if col not in categorical]

df_noncat = pd.DataFrame(imputer.fit_transform(df_raw.loc[:, non_cat]), columns=non_cat)
df_noncat.head()

In [ ]:
for col in missing:
    display(f"for col - {col}:")
    display(df_raw[col].value_counts())

In [ ]:
# impute the missing value with 0
df_raw["HL_tag"].fillna(value=0, inplace=True)
df_raw["AL_tag"].fillna(value=0, inplace=True)

In [ ]:
# remove this col as it's a dummy
df_raw["PC"].value_counts()

In [ ]:
df_raw.drop("PC", axis=1, inplace=True)

In [ ]:
df_raw.HL_tag.value_counts()

In [ ]:
# define columns' categories
nominal = ["C_EDU", "C_HSE", "gn_occ", "HL_tag", "AL_tag", "C_seg"]
ordinal = ["INCM_TYP"]
target_name = "C_seg"
index_col = "C_ID"

In [ ]:
interval_cols = [col for col in df_raw.columns if col not in (nominal + ordinal)]
interval_cols

In [ ]:
# explore overall correlations
df_corr = df_raw.loc[:, [c for c in df_raw.columns if c not in ["AL_tag", "pur_price_avg", "UT_AVE", "MAXUT", "N_FUNDS"]]]
phik_overview = df_corr.phik_matrix(interval_cols=interval_cols)
f = plt.figure(figsize=(15, 15))
plt.matshow(phik_overview, fignum=f.number, cmap="Reds")
plt.xticks(range(df_corr.shape[1]), df_corr.columns, fontsize=8, rotation=90)
plt.yticks(range(df_corr.shape[1]), df_corr.columns, fontsize=8)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=8)
plt.title('Correlation matrix', fontsize=10)

In [ ]:
# C_seg is relatively more correlated with C_AGE, gn_occ, NUM_PRD, CASATD_CNT

In [ ]:
# convert categorical to string
for col in (nominal + ordinal):
    df_raw[col] = df_raw[col].astype(str)

df_raw.head(2)

In [ ]:
# After converting, np.nan has been converted to string as well
df_raw.loc[(df_raw.isin(["nan", "NaN", "NA", "Nan", "Nill", "NAN"])).any(axis=1)]


In [ ]:
# map string back to np.nan
df_raw.replace({"nan": np.nan, "NaN": np.nan, "NAN": np.nan}, inplace=True)

In [ ]:
df_raw.head()

In [ ]:
# prepare list of features for feature engineering
counts = [
    "NUM_PRD",
    "CASATD_CNT",
    "N_FUNDS",
    "ANN_N_TRX",
    ]

values = [
    "MTHCASA",
    "MAXCASA",
    "MINCASA",
    "pur_price_avg",
    "UT_AVE",
    "MAXUT",
    "CC_AVE",
    "MAX_MTH_TRN_AMT",
    "MIN_MTH_TRN_AMT",
    "MTHTD",
    "MAXTD",
    "Asset value",
    "AVG_TRN_AMT",
    "ANN_TRN_AMT",
    "CC_LMT",
    ]

# show the numeric columns not included in feature engineering list
[col for col in df_raw.columns if ((df_raw[col].dtype != "O") and (col not in counts + values))]

In [ ]:
df_raw.head()

In [ ]:
# prepare df for standardization
stand_cols = counts + values + ['C_AGE', 'DRvCR']
df_stand = df_raw.loc[:, stand_cols]
df_raw.drop(stand_cols, axis=1, inplace=True)
df_stand.head()

In [ ]:
# standardize features
scaler = StandardScaler()
feature_arr = scaler.fit_transform(df_stand.values)
df_stand = pd.DataFrame(feature_arr, index=df_stand.index, columns=df_stand.columns)

df_raw = pd.concat([df_raw, df_stand], axis=1)
display("after scaling", df_raw.head())

In [ ]:
# pd.set_option('display.max_rows', None)
# # check avail transform
# ft.list_primitives()[ft.list_primitives().type == "transform"]

In [ ]:
values

In [ ]:
# feature engineering 1
es = ft.EntitySet(id='ft')
es = es.add_dataframe(dataframe_name="ft", dataframe=df_raw.loc[:, values + [index_col]], index=index_col)

features_matrix,feature_names = ft.dfs(
    entityset=es,
    target_dataframe_name='ft',
    trans_primitives=["add_numeric", "subtract_numeric", "divide_numeric",'multiply_numeric'],
    max_depth=2,
    verbose=True)

features_matrix.reset_index(inplace=True)
features_matrix = features_matrix.drop(values, axis=1)

df_raw = df_raw.merge(features_matrix, how='left', on=index_col)

df_raw.head()

In [ ]:
# feature engineering 2
es = ft.EntitySet(id='ft')
es = es.add_dataframe(dataframe_name="ft", dataframe=df_raw.loc[:, counts + [index_col]], index=index_col)

features_matrix,feature_names = ft.dfs(
    entityset=es,
    target_dataframe_name='ft',
    trans_primitives=["add_numeric", "subtract_numeric", "divide_numeric",'multiply_numeric'],
    max_depth=2,
    verbose=True)

features_matrix.reset_index(inplace=True)
features_matrix = features_matrix.drop(counts, axis=1)

df_raw = df_raw.merge(features_matrix, how='left', on=index_col)

df_raw.head()

In [ ]:
df_raw.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# drop the customer_id
df_X = df_raw.drop([index_col, target_name], axis=1)
df_y = df_raw.loc[:, [target_name]]

In [ ]:
# train, val, test split
X, X_test, y, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=1, shuffle=True, stratify=df_y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1, shuffle=True, stratify=y)

display(f"train set size: {X_train.shape}, val set size: {X_val.shape}, test set size: {X_test.shape}")

In [ ]:
# get cols requiring onehotencoding
categorical = [col for col in (ordinal + nominal) if col != target_name]
display(f"the categorical variables that need one hot encoding are: {categorical}")

# apply onehotencoding for categorical variables
enc = OneHotEncoder(handle_unknown='error', sparse_output=False, drop=None)
enc.fit_transform(df_X.loc[:, categorical])
feature_labels = enc.get_feature_names_out()


feature_arr = enc.transform(X[categorical])
cat_X = pd.DataFrame(feature_arr, columns=feature_labels).reset_index(drop=True)
X = pd.concat([X.drop(categorical, axis=1).reset_index(drop=True), cat_X], axis=1)

# for training data
feature_arr = enc.transform(X_train[categorical])
cat_train = pd.DataFrame(feature_arr, columns=feature_labels).reset_index(drop=True)
X_train = pd.concat([X_train.drop(categorical, axis=1).reset_index(drop=True), cat_train], axis=1)
display("training data after onehot encoding:", X_train.head())

feature_arr = enc.transform(X_val[categorical])
cat_val = pd.DataFrame(feature_arr, columns=feature_labels).reset_index(drop=True)
X_val = pd.concat([X_val.drop(categorical, axis=1).reset_index(drop=True), cat_val], axis=1)
display("val data after onehot encoding:", X_val.head())

feature_arr = enc.transform(X_test[categorical])
cat_test = pd.DataFrame(feature_arr, columns=feature_labels).reset_index(drop=True)
X_test = pd.concat([X_test.drop(categorical, axis=1).reset_index(drop=True), cat_test], axis=1)
display("test data after onehot encoding:", X_test.head())

In [ ]:
# impute missing predictor values with np.nan, so that XGBoost will handle them
X = X.replace(np.nan)
X_train = X_train.replace(np.nan)
X_val = X_val.replace(np.nan)
X_test = X_test.replace(np.nan)

In [ ]:
# onehotencoder will encode the np.nan into a new column
nan_cols = [col for col in X.columns if "_nan" in col]
nan_cols

In [ ]:
# as onehot add extra nan column, so we need to remove it
def impute_encoded_col(df: pd.DataFrame, col_lst: list) -> pd.DataFrame:
    for col_n in col_lst:
        key = col_n[:-4]
        index =(df.loc[df[col_n] == 1]).index

        # drop column
        df.drop(col_n, axis=1, inplace=True)

        # impute with np.nan
        df.loc[index, [col for col in df.columns if key in col]] = np.nan

    return df

X = impute_encoded_col(X, nan_cols)

X_train = impute_encoded_col(X_train, nan_cols)
display("train set after imputing:", X_train.sample(1))

X_val = impute_encoded_col(X_val, nan_cols)
display("val set after imputing:", X_val.sample(1))

X_test = impute_encoded_col(X_test, nan_cols)
display("test set after imputing:", X_test.sample(1))

In [ ]:
# # try to drop useless columns based on feature selection
useless = ['C_HSE_OFFICE',
 'ANN_TRN_AMT / AVG_TRN_AMT',
 'C_HSE_COMMERICAL BUILDING',
 'AVG_TRN_AMT / ANN_TRN_AMT',
 'C_HSE_INDUSTRIAL BUILDING',
 'C_HSE_HOTEL/ SERVICE APARTMENT']

X.drop(useless, axis = 1, inplace=True)
X_train.drop(useless, axis = 1, inplace=True)
X_val.drop(useless, axis = 1, inplace=True)
X_test.drop(useless, axis = 1, inplace=True)

In [ ]:
# label encoding for target variable
le = preprocessing.LabelEncoder()

y = pd.DataFrame(le.fit_transform(y[target_name]), columns=["Y"])

y_train = pd.DataFrame(le.transform(y_train[target_name]), columns=["Y"])
display("training after label encoding:", y_train.head())

y_val = pd.DataFrame(le.transform(y_val[target_name]), columns=["Y"])
display("y_val after label encoding:", y_val.head())

y_test = pd.DataFrame(le.transform(y_test[target_name]), columns=["Y"])
display("y_test after label encoding:", y_test.head())

In [ ]:
y_train.values == 0

In [ ]:
# calculate sample weights
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=[0, 1, 1, 1, 0, 1]
)

# parameters
objective='binary:logistic'
booster = "gbtree"
eval_metric="logloss"
early_stopping_rounds = 5

In [ ]:
%%time

# Finetune Hyperparameters
params = {
    "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9],
    "gamma": uniform(0.3, 0.7),
    "max_depth": [8, 9, 10, 11, 12],
    "n_estimators": randint(30, 80),
    "subsample": [0.6, 0.7, 0.8, 0.9],
    "min_child_weight": [1, 1.5, 2, 2.5],
    "eta": [0.3, 0.1, 0.05],
}

k = StratifiedKFold(n_splits=3, shuffle=False)
scoring="f1"

cv_model = xgb.XGBClassifier(
    objective=objective,
    tree_method= "auto",
    eval_metric=eval_metric,
    booster = booster)

search = RandomizedSearchCV(
    cv_model,
    param_distributions=params,
    scoring=scoring,
    random_state=1,
    n_iter=1,  # No. of combinations / fold
    cv=k,
    verbose=1,
    n_jobs=1,
    return_train_score=True,
    refit=False,  # refit by manual as we need to plot the train-val curve to see overfitting problem
    )

search.fit(X_train, y_train, sample_weight=sample_weights)
cv_results =  pd.DataFrame(search.cv_results_).loc[:, ["rank_test_score", "mean_test_score", "params"]].sort_values(by=["rank_test_score"])
cv_results.head(5)

# show the best set of hyperparams
search.best_params_

In [ ]:
# plot train-val curve to ensure no overfitting
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

colsample_bytree = 0.6
n_estimators = 69
tree_method= "auto"

# tuning parameters
eta = 0.3
max_depth = 10
max_leaves = 2 ** max_depth

# prevent overfitting
min_child_weight = 1.5
gamma = 0.45095519663195377
subsample = 0.9


# Use "hist" for constructing the trees, with early stopping enabled.
model = xgb.XGBClassifier(
    early_stopping_rounds=early_stopping_rounds,
    tree_method=tree_method,
    objective=objective,
    n_estimators=n_estimators,
    missing=np.nan,
    eval_metric=eval_metric,
    booster=booster,
    eta=eta,
    max_depth=max_depth,
    max_leaves=max_leaves,
    min_child_weight=min_child_weight,
    gamma=gamma,
    subsample=subsample,
    colsample_bytree=colsample_bytree)

# Fit the model, val sets are used for early stopping.
result = model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], sample_weight=sample_weights, verbose=True)

# check overfitting
results = result.evals_result()
epochs = len(results['validation_0']['logloss'])
x_axis = range(0, epochs)

# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Validation')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()

In [ ]:
# Threshold Moving/Tuning
from numpy import arange
from numpy import argmax

yhat = model.predict_proba(X_val)

# keep probabilities for the positive outcome only
probs = yhat[:, 1]

# define thresholds
thresholds = arange(0, 1, 0.001)

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

# evaluate each threshold
scores = [f1_score(y_val, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)

best_thresh = thresholds[ix]

print('Best performing threshold=%.3f with best F-Score=%.5f' % (best_thresh, scores[ix]))

In [ ]:
# finally train with all traing + val, and change to dart booster
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y
)

colsample_bytree = 0.6
n_estimators = 69
tree_method= "auto"
# booster = "dart"  # update booster to dart
booster = "gbtree"

# tuning parameters
eta = 0.3
max_depth = 10
max_leaves = 2**max_depth

# prevent overfitting
min_child_weight = 1.5
gamma = 0.45095519663195377
subsample = 0.9


# Use "hist" for constructing the trees, with early stopping enabled.
model = xgb.XGBClassifier(
    tree_method=tree_method,
    objective=objective,
    n_estimators=n_estimators,
    missing=np.nan,
    eval_metric=eval_metric,
    booster=booster,
    eta=eta,
    max_depth=max_depth,
    max_leaves=max_leaves,
    min_child_weight=min_child_weight,
    gamma=gamma,
    subsample=subsample,
    colsample_bytree=colsample_bytree)

model.fit(X, y, verbose=True, sample_weight=sample_weights)

In [ ]:
# train
preds = model.predict(X_train)

# 0.9130316495873766
f1_score(y_train, preds)

In [ ]:
# val
preds = model.predict(X_val)

# 0.9130316495873766
f1_score(y_val, preds)

In [ ]:
# predict on test set
preds = model.predict(X_test)

# 0.9196223751544026
f1_score(y_test, preds)

In [ ]:
# predict on test set
prob = model.predict_proba(X_test)[:, 1]

preds = (prob > best_thresh).astype("int")

# 0.9228875406664908
f1_score(y_test, preds)

In [ ]:
# plot the feature importance
feature_imp = pd.DataFrame(sorted(zip(model.feature_names_in_, model.feature_importances_), key=lambda x: x[0], reverse=True), columns=['Feature', 'Value'])
fig, ax = plt.subplots()

data = feature_imp.sort_values(by="Value", ascending=False).head(50)
fig.set_size_inches(18.5, 10.5)

sns.barplot(x="Value", y="Feature", data=data)
ax.set_title('Feature Importance By Weight')
ax.set_xlabel('importance score')
ax.set_ylabel('features')
ax.yaxis.set_visible(True)
ax.xaxis.set_visible(True)
fig.patch.set_facecolor('white')
# plt.savefig(f'../data/feature_importance_by_weight.png', bbox_inches='tight')

In [ ]:
# get the useless col in prediction
useless_cols = feature_imp.sort_values(by="Value", ascending=False).loc[feature_imp.sort_values(by="Value", ascending=False).Value == 0, "Feature"].tolist()
useless_cols

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(X_test)

In [ ]:
shap.summary_plot(shap_values.values, X_test, max_display=10, show=False, plot_type="dot", plot_size=[10,10])
